In [ ]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [ ]:
env=BlackjackEnv()

In [ ]:
def make_epsilon_greedy_policy(Q, epsilon,nA):
    
    def policy_fn(observation):
        
        A= np.ones(nA, dtype=float)*epsilon/nA
        print("np.ones(nA, dtype=float)*epsilon/nA" ,A)
              
        best_action =np.argmax(Q[observation])
        print("np.argmax(Q[oservation]) is ", best_action)
        A[best_action] += (1.0 - epsilon)
        return A 
        print(A)
        
    return policy_fn

              

In [ ]:
def mc_control_epsilon_greedy(env, num_episodes, discount_factor = 1.0, epsilon=0.1):
    
    
    returns_sum = defaultdict(float)
    returns_count=defaultdict(float)
    
    
    Q= defaultdict(lambda:np.zeros(env.action_space.n))
    
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    for i_episode in range(1, num_episodes +1):
        if i_episode % 1000 ==0: 
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        episode =[]
        state=env.reset()
        
        for t in range(100):
            probs = policy(state)
            action=np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            print("episode is added ", episode)
            if done:
                break
            state = next_state
            
        sa_in_episode = set([(tuple(x[0]), x[1]) for x in episode])
        
        print("sa_in_episode = set([tuple(x[0]), x[1]]) for x in episode])",sa_in_episode)
        
        for state, action in sa_in_episode:
            sa_pair=(state, action)
            print("(state,action)",sa_pair)
            first_occurence_idx = next(i for i,x in enumerate(episode)
                                      if x[0] == state and x[1] == action) 
            G= sum([x[2]* (discount_factor**i) for i,x in enumerate(episode[first_occurence_idx:])])
                     
                    
                     
                     #calculate average return for this state over all sampled episodes
                     
            returns_sum[sa_pair] += G
            returns_count[sa_pair] += 1.0 
            
            Q[state][action] = returns_sum[sa_pair] / returns_count[ssdf]
            
            
    return Q, policy
                    
                     
       
        
        
        
        #find all
        
        
           

In [ ]:
Q,policy = mc_control_epsilon_greedy(env,100000, epsilon=0.1)

In [ ]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, actions in Q.items():
    action_value = np.max(actions)
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")